<a href="https://colab.research.google.com/github/Mohamed-Adel-l/Supervised-Learning/blob/main/LogisticRegression_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import keras
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score , confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.preprocessing import LabelEncoder


In [ ]:
rice_dataset_raw = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/Rice_Cammeo_Osmancik.csv")
x=rice_dataset_raw.drop('Class',axis=1)
y=rice_dataset_raw['Class']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(rice_dataset_raw.describe())
print("\nData Shapes:\n" + "-" * 40)
print(f"{'X_train shape':<15}: {X_train.shape}")
print(f"{'X_test shape' :<15}: {X_test.shape}")
print(f"{'y_train shape':<15}: {y_train.shape}")
print(f"{'y_test shape' :<15}: {y_test.shape}")

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

print("\nLabel Encoding Map:\n" + "-" * 40)
for class_name, class_id in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
    print(f"{class_name:<10} → {class_id}")

               Area    Perimeter  Major_Axis_Length  Minor_Axis_Length  \
count   3810.000000  3810.000000        3810.000000        3810.000000   
mean   12667.727559   454.239180         188.776222          86.313750   
std     1732.367706    35.597081          17.448679           5.729817   
min     7551.000000   359.100006         145.264465          59.532406   
25%    11370.500000   426.144753         174.353855          82.731695   
50%    12421.500000   448.852493         185.810059          86.434647   
75%    13950.000000   483.683746         203.550438          90.143677   
max    18913.000000   548.445984         239.010498         107.542450   

       Eccentricity   Convex_Area       Extent  
count   3810.000000   3810.000000  3810.000000  
mean       0.886871  12952.496850     0.661934  
std        0.020818   1776.972042     0.077239  
min        0.777233   7723.000000     0.497413  
25%        0.872402  11626.250000     0.598862  
50%        0.889050  12706.500000     0

In [ ]:
results = {}
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

class Batch_Gradient_Descent:
  def __init__(self, learning_rate=0.001, n_iters=1000):
    self.lr = learning_rate
    self.n_iters = n_iters
  def sigmoid(self, x):
    return 1/(1+np.exp(-x))
  def loss(self, y, y_pred):
    return (-y*np.log(y_pred) - (1-y)*np.log(1-y_pred)).mean()
  def fit(self, X, y):
    self.weights =np.zeros(X.shape[1])
    self.bias = 0
    self.losses = []
    for i in range(self.n_iters):
      z_pred = np.dot(X, self.weights) + self.bias
      y_pred = self.sigmoid(z_pred)
      dw = (1/X.shape[0])*np.dot(X.T, (y_pred-y))
      db = (1/X.shape[0])*np.sum(y_pred-y)
      self.weights -= self.lr*dw
      self.bias -= self.lr*db
      self.losses.append(self.loss(y, y_pred))
      print(f"epoch:{i+1}/{self.n_iters}, loss:{self.losses[-1]}")
  def predict(self, X):
    z_pred = np.dot(X, self.weights) + self.bias
    y_pred = self.sigmoid(z_pred)
    y_pred = np.where(y_pred>0.5, 1, 0)
    return y_pred

In [ ]:
class Mini_batch_gradient_descent:
  def __init__(self, learning_rate=0.001, n_iters=1000, batch_size=32):
    self.lr = learning_rate
    self.n_iters = n_iters
    self.batch_size = batch_size
  def sigmoid(self, x):
    return 1/(1+np.exp(-x))
  def loss(self, y, y_pred):
    return (-y*np.log(y_pred) - (1-y)*np.log(1-y_pred)).mean()
  def fit(self, X, y):
    self.weights =np.zeros(X.shape[1])
    self.bias = 0
    self.losses = []
    n_samples=X.shape[0]
    for i in range(self.n_iters):
     indices = np.arange(n_samples)
     np.random.shuffle(indices)
     X = X[indices]
     y = y[indices]
     for j in range(0, n_samples, self.batch_size):
       X_batch = X[j:j+self.batch_size]
       y_batch = y[j:j+self.batch_size]
       z_pred = np.dot(X_batch, self.weights) + self.bias
       y_pred = self.sigmoid(z_pred)
       dw = (1/self.batch_size)*np.dot(X_batch.T, (y_pred-y_batch))
       db = (1/self.batch_size)*np.sum(y_pred-y_batch)
       self.weights -= self.lr*dw
       self.bias -= self.lr*db
       self.losses.append(self.loss(y_batch, y_pred))
     print(f"epoch:{i+1}/{self.n_iters}, loss:{self.losses[-1]}")
  def predict(self, X):
    y_pred = np.dot(X, self.weights) + self.bias
    y_pred = self.sigmoid(y_pred)
    y_pred = np.where(y_pred>0.5, 1, 0)
    return y_pred

In [ ]:
class Stochastic_gradient_descent:
  def __init__(self, learning_rate=0.001, n_iters=1000):
    self.lr = learning_rate
    self.n_iters = n_iters
  def sigmoid(self, x):
    return 1/(1+np.exp(-x))
  def loss(self, y, y_pred):
    return (-y*np.log(y_pred) - (1-y)*np.log(1-y_pred)).mean()
  def fit(self, X, y):
    self.weights =np.zeros(X.shape[1])
    self.bias = 0
    self.losses = []
    n_samples=X.shape[0]
    for i in range(self.n_iters):
     indices = np.arange(n_samples)
     np.random.shuffle(indices)
     X_shuffled = X[indices]
     y_shuffled= y[indices]
     for j in range(n_samples):
       X_j = X_shuffled[j].reshape(1, -1)
       y_j = y_shuffled[j]
       z_pred = np.dot(X_j, self.weights) + self.bias
       y_pred = self.sigmoid(z_pred)
       dw = np.dot(X_j.T, (y_pred-y_j))
       db = np.sum(y_pred-y_j)
       self.weights -= self.lr*dw
       self.bias -= self.lr*db
       self.losses.append(self.loss(y_j, y_pred))
     print(f"epoch:{i+1}/{self.n_iters}, loss:{self.losses[-1]}")
  def predict(self, X):
    z_pred = np.dot(X, self.weights) + self.bias
    y_pred = self.sigmoid(z_pred)
    y_pred = np.where(y_pred>0.5, 1, 0)
    return y_pred


In [ ]:
model1 = Batch_Gradient_Descent(learning_rate=0.01, n_iters=10)
model1.fit(X_train, y_train)

y_pred = model1.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n" + "-" * 30)
print(cm)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nEvaluation Metrics:\n" + "-" * 30)
print(f"{'Accuracy' :<10}: {acc:.4f}")
print(f"{'Precision':<10}: {prec:.4f}")
print(f"{'Recall'   :<10}: {rec:.4f}")
print(f"{'F1 Score' :<10}: {f1:.4f}")
results['BGD'] = acc

epoch:1/10, loss:0.6931471805599454
epoch:2/10, loss:0.6859784498399223
epoch:3/10, loss:0.6789686275814459
epoch:4/10, loss:0.6721140603671435
epoch:5/10, loss:0.6654111313974577
epoch:6/10, loss:0.6588562648063736
epoch:7/10, loss:0.6524459295717169
epoch:8/10, loss:0.6461766430308731
epoch:9/10, loss:0.6400449740147822
epoch:10/10, loss:0.634047545614724

Confusion Matrix:
------------------------------
[[323  27]
 [ 36 376]]

Evaluation Metrics:
------------------------------
Accuracy  : 0.9173
Precision : 0.9330
Recall    : 0.9126
F1 Score  : 0.9227


In [ ]:
model2 = Mini_batch_gradient_descent(learning_rate=0.01, n_iters=10, batch_size=2)
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n" + "-" * 30)
print(cm)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nEvaluation Metrics:\n" + "-" * 30)
print(f"{'Accuracy' :<10}: {acc:.4f}")
print(f"{'Precision':<10}: {prec:.4f}")
print(f"{'Recall'   :<10}: {rec:.4f}")
print(f"{'F1 Score' :<10}: {f1:.4f}")
results['MBGD'] = acc

epoch:1/10, loss:0.20466251868941504
epoch:2/10, loss:0.03476396557710363
epoch:3/10, loss:0.01644810926420136
epoch:4/10, loss:0.12352480818784094
epoch:5/10, loss:0.220347293070528
epoch:6/10, loss:0.000568692678460896
epoch:7/10, loss:0.01751331325846595
epoch:8/10, loss:0.044985992404816606
epoch:9/10, loss:0.1479109049703373
epoch:10/10, loss:0.008993169907107652

Confusion Matrix:
------------------------------
[[324  26]
 [ 29 383]]

Evaluation Metrics:
------------------------------
Accuracy  : 0.9278
Precision : 0.9364
Recall    : 0.9296
F1 Score  : 0.9330


In [ ]:
model3= Stochastic_gradient_descent(learning_rate=0.01, n_iters=10)
model3.fit(X_train, y_train)
y_pred = model3.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n" + "-" * 30)
print(cm)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\nEvaluation Metrics:\n" + "-" * 30)
print(f"{'Accuracy' :<10}: {acc:.4f}")
print(f"{'Precision':<10}: {prec:.4f}")
print(f"{'Recall'   :<10}: {rec:.4f}")
print(f"{'F1 Score' :<10}: {f1:.4f}")
results['SGD'] = acc

epoch:1/10, loss:1.1741183695467
epoch:2/10, loss:0.007132458306796421
epoch:3/10, loss:0.005755558686656315
epoch:4/10, loss:0.0028311840923905215
epoch:5/10, loss:0.007624223556303398
epoch:6/10, loss:0.0032009374401818637
epoch:7/10, loss:0.10872303370709462
epoch:8/10, loss:0.04006737357247391
epoch:9/10, loss:0.051592705020749786
epoch:10/10, loss:0.04291991826214951

Confusion Matrix:
------------------------------
[[322  28]
 [ 22 390]]

Evaluation Metrics:
------------------------------
Accuracy  : 0.9344
Precision : 0.9330
Recall    : 0.9466
F1 Score  : 0.9398
